# SVD and ICA exploration
This notebook is an attempt to understand the application of independent component analysis (ICA) on top of the output of the singular value decomposition (SVD). 

The code in this notebook was written by Esten Leonardsen, and extended by me.


## 1. Synthethic data creation
To illustrate the concept, we create synthetic data which consists of three underlying sources that we are trying to find with the ICA.

In [ ]:
import math
import numpy as np
import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots

init_notebook_mode(connected=True)

n_samples = 100
n_sources = 3 # MAX 6

sources = [
    np.arange(0, n_samples, dtype=float),
    np.sin(np.arange(0, 10, 10/n_samples)),
    np.asarray([1. if (i // (n_samples//8)) % 2 == 0 else 2. for i in range(n_samples)]),
    np.concatenate([np.arange(0, int(math.floor(n_samples // 2)), dtype=float), 
                    np.arange(int(math.ceil(n_samples // 2)), 0, -1, dtype=float)]),
    np.arange(n_samples, 0, -1, dtype=float) ** 2,
    np.arange(0, 5, 5/n_samples).astype(int).astype(float)
    
]
sources = sources[:n_sources]
sources = np.asarray(sources)
sources = sources.T

def normalize(x):
    x -= np.amin(x)
    x /= np.amax(x)
    
    return x

for i in range(n_sources):
    sources[:,i] = normalize(sources[:, i])

traces = [
    go.Scatter(
        x=np.arange(n_samples),
        y=sources[:,i],
        mode='lines',
        name=f'Source {i}'
    ) 
for i in range(n_sources)]

layout = go.Layout(
    title = {
        'x': 0.5,
        'text': 'Original sources'
    }
)

iplot(go.Figure(traces, layout))

## 2. Place the signal in the the columns of our data matrix.

Create signals that are a linear combination of the three sources given above.

In [18]:
n_idps = 50
noise = 5e-2

idps = []
formulas = []

for i in range(n_idps):
    factors = np.random.uniform(low=0, high=1, size=n_sources)
    idp = np.sum([factors[i] * sources[:,i] for i in range(n_sources)], axis=0)
    idp += np.random.normal(0, noise, size=idp.shape)
    idps.append(idp)
    formulas.append(' + '.join([f'{np.round(factors[i], 1)} * x[{i}]' for i in range(n_sources)]))

idps = np.stack(idps)
idps = idps.T
print(np.shape(idps))
traces = [
    go.Scatter(
        x=np.arange(n_samples),
        y=idps[:, i],
        mode='lines',
        name=formulas[i],
        showlegend=False
    )
for i in range(n_idps)]

layout = go.Layout(
    title = {
        'x': 0.5,
        'text': 'Derived IDPs'
    }
)

iplot(go.Figure(traces, layout))

(100, 50)


Our data matrix, _idps_, is now a n_samples x n_idps matrix. The linear combination of the signal is to be found in each column.

### 2.1 Apply ICA to find the underlying signal in the columns
Now, we will apply ICA onto our data matrix. The FastICA algorithm implemented in python expects the samples to be in the columns.

In [37]:
from sklearn.decomposition import FastICA

n_components = n_sources

ica = FastICA(n_components)
components = ica.fit_transform(idps)
componentsT = ica.fit_transform(idps.T)

figure = make_subplots(2, 1, subplot_titles=['ICA (columns)','ICA (rows)'])
for i in range(n_components):
    figure.add_trace(
        go.Scatter(
            x=np.arange(n_samples),
            y=components[:, i],
            mode='lines',
            name=f'ICA (column) component {i}'
        ),
        row=1,
        col=1
    )
    figure.add_trace(
        go.Scatter(
            x=np.arange(n_samples),
            y=componentsT[:, i],
            mode='lines',
            name=f'ICA (row) component {i}'
        ),
        row=2,
        col=1
    )

iplot(figure)

### 2.2 Apply SVD, then ICA
Now, let us do the singular value decomposition (SVD), and then apply ICA on the output of the left and right singular vectors.

In [38]:
from scipy import linalg

U, s, Vh = linalg.svd(idps)
V = Vh.T

sigma = np.zeros((n_samples,n_idps))
for i in range(min(n_samples,n_idps)):
    sigma[i,i] = s[i]

US = np.dot(U,sigma)
VS = np.dot(V,sigma.T)

ica = FastICA(n_components)
components_us = ica.fit_transform(US)
components_vs = ica.fit_transform(VS)

figure = make_subplots(2, 1, subplot_titles=['US+ICA','VS+ICA'])
for i in range(n_components):
    figure.add_trace(
        go.Scatter(
            x=np.arange(n_samples),
            y=components_us[:, i],
            mode='lines',
            name=f'US+ICA component {i}'
        ),
        row=1,
        col=1
    )
    figure.add_trace(
        go.Scatter(
            x=np.arange(n_samples),
            y=components_vs[:, i],
            mode='lines',
            name=f'VS+ICA component {i}'
        ),
        row=2,
        col=1
    )

iplot(figure)

We see that if the signal is in the columns in our data matrix, *idps* , we will retrieve it in the U matrix in the SVD, but not in the V.

## 3. Place the signal in the rows of our data matrix
Now, let us transpose our data matrix *idps*, so that the synthetic signal generated is in each row, and run the same analysis as above

In [39]:
A = idps.T
traces = [
    go.Scatter(
        x=np.arange(n_samples),
        y=A[:, i],
        mode='lines',
        name=formulas[i],
        showlegend=False
    )
for i in range(n_idps)]

layout = go.Layout(
    title = {
        'x': 0.5,
        'text': 'Derived IDPs'
    }
)

iplot(go.Figure(traces, layout))

The plotted columns now look random. As the signal is along the rows. Let us try to apply ICA the same way as we did above. Now, we should expect that the ICA should not find the sources as they are not present in the columns.

### 3.1 Apply ICA on the columns

In [40]:
from sklearn.decomposition import FastICA

n_components = n_sources

ica = FastICA(n_components)
components = ica.fit_transform(A)
componentsT = ica.fit_transform(A.T)

figure = make_subplots(2, 1, subplot_titles=['ICA (columns)','ICA (rows)'])
for i in range(n_components):
    figure.add_trace(
        go.Scatter(
            x=np.arange(n_samples),
            y=components[:, i],
            mode='lines',
            name=f'ICA (column) component {i}'
        ),
        row=1,
        col=1
    )
    figure.add_trace(
        go.Scatter(
            x=np.arange(n_samples),
            y=componentsT[:, i],
            mode='lines',
            name=f'ICA (row) component {i}'
        ),
        row=2,
        col=1
    )

iplot(figure)

As expected, we do not find any signal in the columns of our data matrix, as it does not exist.

### 3.2 Apply SVD, then ICA

In [41]:
from scipy import linalg

U, s, Vh = linalg.svd(A)
V = Vh.T

sigma = np.zeros((n_samples,n_idps)).T
for i in range(n_idps):
    sigma[i,i] = s[i]

US = np.dot(U,sigma)
VS = np.dot(V,sigma.T)

ica = FastICA(n_components)
components_us = ica.fit_transform(US)
components_vs = ica.fit_transform(VS)

from plotly.subplots import make_subplots

figure = make_subplots(2, 1, subplot_titles=['US+ICA','VS+ICA'])
for i in range(n_components):
    figure.add_trace(
        go.Scatter(
            x=np.arange(n_samples),
            y=components_us[:, i],
            mode='lines',
            name=f'US+ICA component {i}'
        ),
        row=1,
        col=1
    )
    figure.add_trace(
        go.Scatter(
            x=np.arange(n_samples),
            y=components_vs[:, i],
            mode='lines',
            name=f'VS+ICA component {i}'
        ),
        row=2,
        col=1
    )

iplot(figure)